In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding,GRU
from keras.models import Sequential

In [2]:
import pandas as pd
import numpy as np
all_data=pd.read_csv('WikiLarge_Train.csv')
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, dev_index in sss.split(all_data, all_data['label']):
    strat_train_set=all_data.iloc[train_index]
    strat_dev_set=all_data.iloc[dev_index]
strat_dev_set['label'].value_counts()/len(strat_dev_set)

0    0.5
1    0.5
Name: label, dtype: float64

In [3]:
all_data.head()

,original_text,label
0,There is manuscript evidence that Austen conti...,1
1,"In a remarkable comparative analysis , Mandaea...",1
2,"Before Persephone was released to Hermes , who...",1
3,Cogeneration plants are commonly found in dist...,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1


In [4]:
all_texts=all_data['original_text'].tolist()
all_labels=all_data['label'].tolist()
MAX_SEQUENCE_LENGTH = 50    #the max seqeunce length
EMBEDDING_DIM = 200    #the size of embedding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(all_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Shape of data tensor: (416768, 50)
Shape of label tensor: (416768, 2)


In [5]:
testing=pd.read_csv('WikiLarge_Test.csv')
all_test=list(testing['original_text'])


In [32]:
len(all_test)

119092

In [33]:
sequences_test = tokenizer.texts_to_sequences(all_test)
X_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [10]:
X_train=data[train_index]
y_train=labels[train_index]
X_dev=data[dev_index]
y_dev=labels[dev_index]

In [23]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 200)           28541000  
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                12864     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130       
Total params: 28,874,794
Trainable params: 28,874,794
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
print(model.metrics_names)
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=2, batch_size=128)
model.save('lstm.h5')

[]
Epoch 1/2
2605/2605 [==============================] - 4035s 2s/step - loss: 0.5988 - acc: 0.6696 - val_loss: 0.5508 - val_acc: 0.7096
Epoch 2/2
2605/2605 [==============================] - 1992s 765ms/step - loss: 0.5307 - acc: 0.7268 - val_loss: 0.5453 - val_acc: 0.7142


In [35]:
predict_results=model.predict(X_test)

In [36]:
len(X_test)

119092

In [37]:
len(predict_results)

119092

In [38]:
result_1=[]
result_2=[]
for i in range(len(predict_results)):
    if predict_results[i][0]>predict_results[i][1]:
        result_1.append(int(0))
        
        
    else:
        result_1.append(int(1))
        

In [39]:
len(result_1)

119092

In [40]:
result_1=pd.DataFrame({'id':list(testing['id']), 'label':result_1})
result_1.to_csv('results_LSTM1.csv',index=None)